# Naver Dataset 전처리
- 긍정(1), 부정(0) 분석

## [1] 모듈 로딩 및 데이터 준비 
---

In [1]:
# 모듈 로딩
from Korpora import Korpora # open Korean Dataset
from konlpy.tag import *    # 형태소 분석기
import spacy                # 형태소 분석기 

from torchtext.data.utils import get_tokenizer # pytorch 자연어 처리 모듈 
from torchtext.vocab import * # pytorch 자연어 처리 단어사전 관련 모듈 
from torch.utils.data import Dataset, DataLoader # pytorch Dataset 관련 모듈
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd 
import numpy as np

c:\Users\KDP-35\anaconda3\envs\TEXT_018_230_38\lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\KDP-35\anaconda3\envs\TEXT_018_230_38\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\KDP-35\anaconda3\envs\TEXT_018_230_38\lib\site-packages\torchtext\utils.py:4: UserWarning:

In [2]:
# 데이터 로딩
nsmc = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-35\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [3]:
nsmc.test[0]

LabeledSentence(text='굳 ㅋ', label=1)

In [4]:
# test 데이터셋을 DF로 로딩 
nsmc_df = pd.DataFrame(nsmc.test)
nsmc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [5]:
nsmc_df

,text,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


## [2] 구두점 및 불용어 제거 
---

In [6]:
# 언어 모델 모델 설정 
LANG_MODEL = 'ko_core_news_lg'

In [7]:
# 한국어 분석기 생성 
nlp = spacy.load(LANG_MODEL)

In [8]:
my_tokens = []

for text in nsmc_df['text']:
    sentence = []
    doc = nlp(text)

    for token in doc:
        if (not token.is_punct) and (not token.is_stop):
            sentence.append(str(token))
            # 구두점과 불용어 아닌 것들은 다 넣음
    my_tokens.append(sentence)
    

In [9]:
my_tokens[:100]

[['굳', 'ㅋ'],
 ['GDNTOPCLASSINTHECLUB'],
 ['뭐야', '평점들은', '나쁘진', '않지만', '10점', '짜리는', '더더욱', '아니잖아'],
 ['지루하지는', '않은데', '완전', '막장임', '돈주고', '보기에는'],
 ['3D만',
  '아니었어도',
  '별',
  '다섯',
  '개',
  '줬을텐데',
  '왜',
  '3D로',
  '나와서',
  '제',
  '심기를',
  '불편하게',
  '하죠'],
 ['음악이', '주가', '된', '최고의', '음악영화'],
 ['진정한', '쓰레기'],
 ['마치', '미국애니에서', '튀어나온듯한', '창의력없는', '로봇디자인부터가', '고개를', '젖게한다'],
 ['갈수록',
  '개판되가는',
  '중국영화',
  '유치하고',
  '내용없음',
  '폼잡다',
  '끝남',
  '말도안되는',
  '무기에',
  '유치한cg남무',
  '아',
  '그립다',
  '동사서독같은',
  '영화가',
  '이건',
  '3류아류작이다'],
 ['이별의', '아픔뒤에', '찾아오는', '새로운', '인연의', '기쁨', 'But', '모든', '사람이', '그렇지는', '않네'],
 ['괜찮네요오랜만포켓몬스터잼밌어요'],
 ['한국독립영화의', '한계', '그렇게', '아버지가', '된다와', '비교됨'],
 ['청춘은',
  '아름답다',
  '아름다움은',
  '이성을',
  '흔들어',
  '놓는다',
  '찰나의',
  '아름다움을',
  '포착한',
  '섬세하고',
  '아름다운',
  '수채화같은',
  '퀴어영화이다'],
 ['눈에', '보이는', '반전이었지만', '영화의', '흡인력은', '사라지지', '않았다'],
 ['스토리',
  '연출',
  '연기',
  '비주얼',
  '영화의',
  '기본',
  '조차',
  '안된',
  '영화에',
  '무슨',
  '평을',
  '해',
  '이런',
  '영화',
  '찍고도',
  

In [10]:
# 토큰을 키로 해서 빈도수 저장 

token_freqs = {}

# 라인(줄) 별 토큰을 읽어서 빈도 체크
for tokenList in my_tokens: 
    for token in tokenList:
        # 토큰 키가 존재하지 않으면 키로 추가 후 1로 설정 
        if token not in token_freqs:
            token_freqs[token] = 1 
            
        # 이미 존재하는 토큰 키이면 값을 1 증가 
        else: 
            token_freqs[token] += 1 

In [11]:
token_freqs

{'굳': 63,
 'ㅋ': 389,
 'GDNTOPCLASSINTHECLUB': 1,
 '뭐야': 105,
 '평점들은': 2,
 '나쁘진': 7,
 '않지만': 39,
 '10점': 300,
 '짜리는': 2,
 '더더욱': 6,
 '아니잖아': 22,
 '지루하지는': 4,
 '않은데': 21,
 '완전': 573,
 '막장임': 2,
 '돈주고': 62,
 '보기에는': 27,
 '3D만': 2,
 '아니었어도': 6,
 '별': 255,
 '다섯': 5,
 '개': 115,
 '줬을텐데': 2,
 '왜': 1129,
 '3D로': 15,
 '나와서': 94,
 '제': 67,
 '심기를': 1,
 '불편하게': 9,
 '하죠': 4,
 '음악이': 47,
 '주가': 1,
 '된': 143,
 '최고의': 760,
 '음악영화': 7,
 '진정한': 95,
 '쓰레기': 464,
 '마치': 63,
 '미국애니에서': 1,
 '튀어나온듯한': 1,
 '창의력없는': 1,
 '로봇디자인부터가': 1,
 '고개를': 5,
 '젖게한다': 1,
 '갈수록': 138,
 '개판되가는': 1,
 '중국영화': 4,
 '유치하고': 70,
 '내용없음': 2,
 '폼잡다': 1,
 '끝남': 12,
 '말도안되는': 24,
 '무기에': 1,
 '유치한cg남무': 1,
 '아': 632,
 '그립다': 26,
 '동사서독같은': 1,
 '영화가': 839,
 '이건': 724,
 '3류아류작이다': 1,
 '이별의': 3,
 '아픔뒤에': 1,
 '찾아오는': 3,
 '새로운': 81,
 '인연의': 2,
 '기쁨': 2,
 'But': 4,
 '모든': 204,
 '사람이': 126,
 '그렇지는': 3,
 '않네': 12,
 '괜찮네요오랜만포켓몬스터잼밌어요': 1,
 '한국독립영화의': 1,
 '한계': 16,
 '그렇게': 183,
 '아버지가': 13,
 '된다와': 1,
 '비교됨': 2,
 '청춘은': 3,
 '아름답다': 60,
 '아름다움은': 2

In [12]:
# 빈도가 높게 나오는 순서대로 단어 정렬 

sorted_tokens = sorted(token_freqs.items(), key=lambda x:x[1], reverse=True)
# (k,v)순서대로 저장되므로 1번 원소는 value
sorted_tokens

[('영화', 6096),
 ('너무', 2871),
 ('정말', 2791),
 ('진짜', 2303),
 ('그냥', 1182),
 ('왜', 1129),
 ('이런', 1116),
 ('영화를', 1042),
 ('다', 906),
 ('영화는', 890),
 ('보고', 870),
 ('영화가', 839),
 ('봤는데', 838),
 ('내가', 768),
 ('본', 767),
 ('최고의', 760),
 ('이건', 724),
 ('ㅋㅋ', 719),
 ('없는', 682),
 ('~', 672),
 ('없다', 655),
 ('평점', 646),
 ('드라마', 645),
 ('이렇게', 640),
 ('아', 632),
 ('이거', 628),
 ('참', 616),
 ('이게', 613),
 ('있는', 578),
 ('완전', 573),
 ('평점이', 573),
 ('다시', 552),
 ('연기', 551),
 ('내', 542),
 ('좋은', 536),
 ('역시', 520),
 ('보는', 518),
 ('♥', 517),
 ('난', 511),
 ('많이', 491),
 ('스토리', 488),
 ('최고', 487),
 ('없고', 478),
 ('ㅠㅠ', 472),
 ('재밌게', 470),
 ('하는', 468),
 ('쓰레기', 464),
 ('보면', 419),
 ('꼭', 408),
 ('ㅋㅋㅋ', 407),
 ('아깝다', 405),
 ('ㅡㅡ', 390),
 ('ㅋ', 389),
 ('영화다', 380),
 ('하지만', 371),
 ('마지막', 364),
 ('무슨', 359),
 ('볼', 355),
 ('끝까지', 340),
 ('뭐', 339),
 ('만든', 333),
 ('작품', 333),
 ('별로', 332),
 ('솔직히', 324),
 ('같은', 323),
 ('가장', 322),
 ('그래도', 319),
 ('대한', 310),
 ('지금', 306),
 ('와', 304),
 ('10점'

In [17]:
sorted_tokens[0][0]

'영화'

## [3] 단어 사전 
---

In [26]:
# 단어 사전 생성 및 초기화 
# 특수 토큰 : 'PAD', 'OOV' 또는 'UNK' 

PAD_TOKEN , OOV_TOKEN = 'PAD','OOV'
vocab = {PAD_TOKEN:0, OOV_TOKEN:1}

In [27]:
# 데이터 코퍼스에서 추출된 토큰(단어)들 
for idx, tk in enumerate(sorted_tokens,2) : # 2부터 시작 
    vocab[tk[0]] = idx


import itertools

for key, value in itertools.islice(vocab.items(), 100):
    print(key, value)

PAD 0
OOV 1
영화 2
너무 3
정말 4
진짜 5
그냥 6
왜 7
이런 8
영화를 9
다 10
영화는 11
보고 12
영화가 13
봤는데 14
내가 15
본 16
최고의 17
이건 18
ㅋㅋ 19
없는 20
~ 21
없다 22
평점 23
드라마 24
이렇게 25
아 26
이거 27
참 28
이게 29
있는 30
완전 31
평점이 32
다시 33
연기 34
내 35
좋은 36
역시 37
보는 38
♥ 39
난 40
많이 41
스토리 42
최고 43
없고 44
ㅠㅠ 45
재밌게 46
하는 47
쓰레기 48
보면 49
꼭 50
ㅋㅋㅋ 51
아깝다 52
ㅡㅡ 53
ㅋ 54
영화다 55
하지만 56
마지막 57
무슨 58
볼 59
끝까지 60
뭐 61
만든 62
작품 63
별로 64
솔직히 65
같은 66
가장 67
그래도 68
대한 69
지금 70
와 71
10점 72
영화의 73
봐도 74
ㅎㅎ 75
말이 76
재미 77
넘 78
있다 79
내내 80
연기도 81
어떻게 82
좋다 83
전혀 84
연기가 85
중 86
뭔가 87
같다 88
많은 89
봤다 90
아주 91
할 92
최악의 93
않고 94
아니라 95
모두 96
듯 97
재미없다 98
별 99


In [52]:
# 텍스트 문장 ==> 수치화 [인코딩]
encoding_data = []

for tokenList in my_tokens:
    # 1개 문장 인코딩
    sent = []
    for token in tokenList:
        sent.append(vocab[token])

    # 인코딩 된 문장 저장 
    encoding_data.append(sent)

for i in range(100):
    print(f'문장 : {my_tokens[i]}')
    print(f'==> 인코딩 : {encoding_data[i]}')

문장 : ['굳', 'ㅋ']
==> 인코딩 : [573, 54]
문장 : ['GDNTOPCLASSINTHECLUB']
==> 인코딩 : [29212]
문장 : ['뭐야', '평점들은', '나쁘진', '않지만', '10점', '짜리는', '더더욱', '아니잖아']
==> 인코딩 : [315, 16912, 5625, 997, 72, 16913, 6445, 1823]
문장 : ['지루하지는', '않은데', '완전', '막장임', '돈주고', '보기에는']
==> 인코딩 : [9248, 1895, 31, 16914, 587, 1473]
문장 : ['3D만', '아니었어도', '별', '다섯', '개', '줬을텐데', '왜', '3D로', '나와서', '제', '심기를', '불편하게', '하죠']
==> 인코딩 : [16915, 6446, 99, 7570, 289, 16916, 7, 2667, 356, 532, 29213, 4434, 9249]
문장 : ['음악이', '주가', '된', '최고의', '음악영화']
==> 인코딩 : [800, 29214, 223, 17, 5626]
문장 : ['진정한', '쓰레기']
==> 인코딩 : [353, 48]
문장 : ['마치', '미국애니에서', '튀어나온듯한', '창의력없는', '로봇디자인부터가', '고개를', '젖게한다']
==> 인코딩 : [574, 29215, 29216, 29217, 29218, 7571, 29219]
문장 : ['갈수록', '개판되가는', '중국영화', '유치하고', '내용없음', '폼잡다', '끝남', '말도안되는', '무기에', '유치한cg남무', '아', '그립다', '동사서독같은', '영화가', '이건', '3류아류작이다']
==> 인코딩 : [233, 29220, 9250, 511, 16917, 29221, 3358, 1662, 29222, 29223, 26, 1534, 29224, 13, 18, 29225]
문장 : ['이별의', '아픔뒤에', '찾아오는', '새로운', '인연의', '기쁨

In [49]:
# 패딩 추가

# 기준 길이 

len_list = []
for tokenList in my_tokens:
    len_list.append(len(tokenList))

MAX_LENGTH = max(len_list)
MAX_LENGTH

38

In [54]:
encoding_data_padding = []

for tokenList in my_tokens:
    # 1개 문장 인코딩
    sent_padding = []
    for token in tokenList:
        sent_padding.append(vocab[token])

    # 인코딩 된 문장 저장 
    encoding_data_padding.append(sent)

# 가장 긴 문장 길이 기준 패딩
for sent in encoding_data_padding:
    cur_length = len(sent)
    if cur_length < MAX_LENGTH:
        print(cur_length, MAX_LENGTH-cur_length)
        sent = sent + ([0]*(MAX_LENGTH-cur_length))


for i in range(100):
    print(f'문장 : {my_tokens[i]}')
    print(f'==> 인코딩 : {encoding_data_padding[i]}')


문장 : ['굳', 'ㅋ']
==> 인코딩 : [5141, 1079, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
문장 : ['GDNTOPCLASSINTHECLUB']
==> 인코딩 : [5141, 1079, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
문장 : ['뭐야', '평점들은', '나쁘진', '않지만', '10점', '짜리는', '더더욱', '아니잖아']
==> 인코딩 : [5141, 1079, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
문장 : ['지루하지는', '않은데', '완전', '막장임', '돈주고', '보기에는']
==> 인코딩 : [5141, 1079, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
문장 : ['3D만', '아니었어도', '별', '다섯', '개', '줬을텐데', '왜', '3D로', '나와서', '제', '심기를', '불편하게', '하죠']
==> 인코딩 : [5141, 1079, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
문장 : ['음악이', '주가', '된', '최고의', '음악영화']
==> 인코딩 : [5141, 1079, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 